In [0]:
# !unzip -uq "/content/drive/My Drive/roi.zip" -d "/content/drive/My Drive/roi_sample"

In [3]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob

In [4]:
path_plus='roi_sample/E01' # 무표정
path_zero='roi_sample/E02' # 웃음
path_minus='roi_sample/E03' # 찡그림
# path_unkno='image/unknown'

la=len(os.listdir(path_zero))
lb=len(os.listdir(path_plus))
lc=len(os.listdir(path_minus))
# lc=len(os.listdir(path_unkno)) 

print('zero 경로에 저장된 파일의 개수:'+str(la))
print('plus 경로에 저장된 파일의 개수:'+str(lb))
print('minus 경로에 저장된 파일의 개수:'+str(lc))

zero 경로에 저장된 파일의 개수:2880
plus 경로에 저장된 파일의 개수:2880
minus 경로에 저장된 파일의 개수:2880


In [5]:
xsize=100
ysize=100

In [6]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [7]:
read_dir(path_zero, 0) #'plus emotion')
read_dir(path_plus, 1) #'zero emotion')
read_dir(path_minus, 2) #'minus emotion')

In [8]:
x=np.array(x)
y=np.array(y)

In [9]:
print(x.shape)

(8640, 100, 100, 3)


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

input_shape = (xsize,ysize, 3)

batch_size = 32
num_classes = 2
epochs = 15

In [11]:
print(x_test.shape)
print(y_test.shape)

(864, 100, 100, 3)
(864,)


In [12]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.applications.xception import Xception 

input_shape = (100, 100, 3)
base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False # feature 고정

x = Flatten()(base_model.output)
output = Dense(3, activation='softmax', name='predictions')(x)
model = Model(inputs=base_model.input, output=output)

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 49, 49, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 49, 49, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 49, 49, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  # This is added back by InteractiveShellApp.init_path()


In [14]:
model.fit(x_train, y_train, validation_split=0.2, 
          epochs=20, batch_size=100, verbose=1) # DenseNet에 비해 속도가 빠름

acc = model.evaluate(x_test, y_test)
print(acc)


Train on 6220 samples, validate on 1556 samples
Epoch 1/20
6220/6220 [==============================] - 36s 6ms/step - loss: 0.7803 - accuracy: 0.6265 - val_loss: 1.4599 - val_accuracy: 0.9030
Epoch 2/20
6220/6220 [==============================] - 24s 4ms/step - loss: 0.1095 - accuracy: 0.9616 - val_loss: 5.7244 - val_accuracy: 0.7481
Epoch 3/20
6220/6220 [==============================] - 24s 4ms/step - loss: 0.0243 - accuracy: 0.9926 - val_loss: 0.2485 - val_accuracy: 0.9492
Epoch 4/20
6220/6220 [==============================] - 24s 4ms/step - loss: 0.0286 - accuracy: 0.9921 - val_loss: 0.3471 - val_accuracy: 0.9531
Epoch 5/20
6220/6220 [==============================] - 25s 4ms/step - loss: 0.0187 - accuracy: 0.9945 - val_loss: 0.0671 - val_accuracy: 0.9865
Epoch 6/20
6220/6220 [==============================] - 25s 4ms/step - loss: 0.0067 - accuracy: 0.9974 - val_loss: 0.0135 - val_accuracy: 0.9955
Epoch 7/20
6220/6220 [==============================] - 25s 4ms/step - loss: 0.02

In [25]:
model.save('save/Xception_freezing(full).h5')
print('모델 저장이 완료되었습니다')

모델 저장이 완료되었습니다


In [15]:
test_path = "test_dataset/Happy/Happy" # 기쁨만 모인 test set
test_path2 = "test_dataset/Neutral/Neutral" # 무표정만 모인 test set 

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
            pass

In [16]:
# test set 기쁨
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

In [18]:
# test set(무표정)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

In [17]:
# 예측(test set 기쁨)
co=0
for i in range(200):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 1:
        co+=1

print('\n',co/200.)


20200200000000000000002020000000000002000020200202000000000000022020020002002000000000000020000000002020002000200002000000000022000002000000022000000020000200000000020022202000000000020020000202220000
 0.0


In [19]:
# 예측(test set 무표정)
co=0
for i in range(200):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n',co/200.)

20200200000000000000002020000000000002000020200202000000000000022020020002002000000000000020000000002020002000200002000000000022000002000000022000000020000200000000020022202000000000020020000202220000
 0.8


In [0]:
# 기쁨 정확도 : 0 %
# 무표정 정확도 : 80 % 

# 역시 무표정 판별률이 더 높다!